In [1]:
import os
from typing import Optional, Tuple

import gym
import numpy as np
import torch
from tianshou.data import Collector, VectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.env.pettingzoo_env import PettingZooEnv
from tianshou.policy import BasePolicy, DQNPolicy, MultiAgentPolicyManager, RandomPolicy
from tianshou.trainer import offpolicy_trainer
from tianshou.utils.net.common import Net

from pettingzoo.classic import tictactoe_v3

In [2]:
env = tictactoe_v3.env(render_mode="human")
env = PettingZooEnv(env)
policies = MultiAgentPolicyManager([RandomPolicy(), RandomPolicy()], env)
env = DummyVectorEnv([lambda: env])
collector = Collector(policies, env)
result = collector.collect(n_episode=1, render=0.1)

     |     |     
  -  |  -  |  X  
_____|_____|_____
     |     |     
  -  |  -  |  -  
_____|_____|_____
     |     |     
  -  |  -  |  -  
     |     |     
     |     |     
  -  |  -  |  X  
_____|_____|_____
     |     |     
  -  |  -  |  -  
_____|_____|_____
     |     |     
  -  |  -  |  -  
     |     |     
     |     |     
  -  |  -  |  X  
_____|_____|_____
     |     |     
  -  |  -  |  -  
_____|_____|_____
     |     |     
  O  |  -  |  -  
     |     |     
     |     |     
  -  |  -  |  X  
_____|_____|_____
     |     |     
  -  |  -  |  -  
_____|_____|_____
     |     |     
  O  |  -  |  -  
     |     |     
     |     |     
  -  |  X  |  X  
_____|_____|_____
     |     |     
  -  |  -  |  -  
_____|_____|_____
     |     |     
  O  |  -  |  -  
     |     |     
     |     |     
  -  |  X  |  X  
_____|_____|_____
     |     |     
  -  |  -  |  -  
_____|_____|_____
     |     |     
  O  |  -  |  -  
     |     |     
     |     |     
  O  |  X 

In [3]:
def _get_agents(
    agent_learn: Optional[BasePolicy] = None,
    agent_opponent: Optional[BasePolicy] = None,
    optim: Optional[torch.optim.Optimizer] = None,
) -> Tuple[BasePolicy, torch.optim.Optimizer, list]:
    env = _get_env()
    observation_space = (
        env.observation_space["observation"]
        if isinstance(env.observation_space, gym.spaces.Dict)
        else env.observation_space
    )
    if agent_learn is None:
        # model
        net = Net(
            state_shape=observation_space["observation"].shape
            or observation_space["observation"].n,
            action_shape=env.action_space.shape or env.action_space.n,
            hidden_sizes=[128, 128, 128, 128],
            device="cuda" if torch.cuda.is_available() else "cpu",
        ).to("cuda" if torch.cuda.is_available() else "cpu")
        if optim is None:
            optim = torch.optim.Adam(net.parameters(), lr=1e-4)
        agent_learn = DQNPolicy(
            model=net,
            optim=optim,
            discount_factor=0.9,
            estimation_step=3,
            target_update_freq=320,
        )

    if agent_opponent is None:
        agent_opponent = RandomPolicy()

    agents = [agent_opponent, agent_learn]
    policy = MultiAgentPolicyManager(agents, env)
    return policy, optim, env.agents

In [4]:
def _get_env():
    """This function is needed to provide callables for DummyVectorEnv."""
    return PettingZooEnv(tictactoe_v3.env())

In [ ]:
# ======== Step 1: Environment setup =========
train_envs = DummyVectorEnv([_get_env for _ in range(10)])
test_envs = DummyVectorEnv([_get_env for _ in range(10)])

# seed
seed = 1
np.random.seed(seed)
torch.manual_seed(seed)
train_envs.seed(seed)
test_envs.seed(seed)

# ======== Step 2: Agent setup =========
policy, optim, agents = _get_agents()

# ======== Step 3: Collector setup =========
train_collector = Collector(
    policy,
    train_envs,
    VectorReplayBuffer(200000, len(train_envs)),
    exploration_noise=True,
)
test_collector = Collector(policy, test_envs, exploration_noise=True)
# policy.set_eps(1)
train_collector.collect(n_step=64 * 10)  # batch size * training_num

# ======== Step 4: Callback functions setup =========
def save_best_fn(policy):
    model_save_path = os.path.join("log", "rps", "dqn", "policy.pth")
    os.makedirs(os.path.join("log", "rps", "dqn"), exist_ok=True)
    torch.save(policy.policies[agents[1]].state_dict(), model_save_path)

def stop_fn(mean_rewards):
    return mean_rewards >= 0.8

def train_fn(epoch, env_step):
    policy.policies[agents[1]].set_eps(0.1)

def test_fn(epoch, env_step):
    policy.policies[agents[1]].set_eps(0.05)

def reward_metric(rews):
    return rews[:, 1]

# ======== Step 5: Run the trainer =========
result = offpolicy_trainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=500,
    step_per_epoch=1000,
    step_per_collect=1,
    episode_per_test=5000,
    batch_size=2,
    train_fn=train_fn,
    test_fn=test_fn,
    stop_fn=stop_fn,
    save_best_fn=save_best_fn,
    update_per_step=0.1,
    test_in_train=False,
    reward_metric=reward_metric,
)

# return result, policy.policies[agents[1]]
print(f"\n==========Result==========\n{result}")
print("\n(the trained policy can be accessed via policy.policies[agents[1]])")

Epoch #1:   0%|                                                                                | 0/1000 [00:00<?, ?it/s]/home/cfs/mh2004/anaconda3/lib/python3.9/site-packages/tianshou/data/collector.py:255: UserWarning: n_step=1 is not a multiple of #env (10), which may cause extra transitions collected into the buffer.
  warnings.warn(
Epoch #1: 1001it [00:01, 784.81it/s, env_step=1000, len=5, n/ep=1, n/st=10, player_2/loss=0.182, rew=-1.00]             


Epoch #1: test_reward: -0.171000 ± 0.925721, best_reward: -0.171000 ± 0.925721 in #1


Epoch #2: 1001it [00:01, 769.68it/s, env_step=2000, len=9, n/ep=1, n/st=10, player_2/loss=0.229, rew=0.00]              


Epoch #2: test_reward: -0.155000 ± 0.936042, best_reward: -0.155000 ± 0.936042 in #2


Epoch #3: 1001it [00:01, 790.12it/s, env_step=3000, len=9, n/ep=1, n/st=10, player_2/loss=0.329, rew=0.00]              


...

Epoch #449: 1001it [00:01, 811.20it/s, env_step=449000, len=7, n/ep=1, n/st=10, rew=-1.00]                              


Epoch #449: test_reward: 0.422000 ± 0.892589, best_reward: 0.494000 ± 0.842831 in #47


Epoch #450: 1001it [00:01, 790.61it/s, env_step=450000, len=6, n/ep=2, n/st=10, rew=0.00]                               


Epoch #450: test_reward: 0.386200 ± 0.905014, best_reward: 0.494000 ± 0.842831 in #47


Epoch #451: 1001it [00:01, 821.34it/s, env_step=451000, len=6, n/ep=0, n/st=10, player_2/loss=0.160, rew=0.00]          
